# PokéData: A Data-Driven Victory

__Context:__ Professor Oak's hypothesis is that Pokémon have been getting stronger with each generation however some of his colleagues seem to disagree. With the Pokémon championship battle coming closer, he wants to use this event to test his theory. He tasks a promising young trainer to fight the undefeated champion Cynthia with a carefully constructed team of 6 pokemon and bring an end to her win streak.